In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
import numpy as np
import pywt
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Preprocessing Pipeline

In [3]:
class PreprocessingPipeline:
    def __init__(self, eeg_raw, channels=None):
        self.eeg_raw = eeg_raw
        self.raw = eeg_raw.copy()

        if channels is None:
            self.channels = self.raw.ch_names
        else:
            self.channels = channels
            self.raw.pick_channels(channels)
            
    def filtering(self, raw, l_freq, h_freq, notch_freq=None):
        """
        Apply bandpass filtering and notch filtering for EEG data.

        Args:
            raw (mne.io.Raw): Raw instance of the data.
            l_freq (float): Lower frequency cutoff for bandpass filter (Hz).
            h_freq (float): Higher frequency cutoff for bandpass filter (Hz).
            notch_freq (float or None): Frequency for notch filtering (Hz). Default is None.

        Returns:
            mne.io.Raw: The filtered EEG data.
        """
        # Apply notch filtering
        if notch_freq is not None and l_freq <= notch_freq <= h_freq:
            self.raw.notch_filter(notch_freq)

        # Apply bandpass filter
        self.raw.filter(l_freq=l_freq, h_freq=h_freq)
        
        return raw
    
    def resample(self, raw, sfreq):
        """
        Resample the EEG data to a new sampling frequency.

        Args:
            raw (mne.io.Raw): Raw instance of the data.
            sfreq (float): The new sampling frequency (in Hz).

        Returns:
            mne.io.Raw: The resampled EEG data.
        """
        self.raw.resample(sfreq, npad='auto')
        
        return self.raw

    def artifact_removal(self, raw, n_components, random_state, tstep=1, ecg_ch=None, eog_ch=None):
        
        """
        Apply ICA to identify and repair artifacts in the EEG data.
        
        Args:
        raw (mne.io.Raw): Raw instance of the data.
        n_components (int): The number of independent components to compute for ICA.
        random_state (int or None): Random state for ICA.
        tstep (float): Time step to segment EEG data (default is 1 second).
        ecg_ch (str or list): Name(s) of ECG channel(s) for ECG artifact repair. Default is None.
        eog_ch (str or list): Name(s) of EOG channel(s) for EOG artifact repair. Default is None.
        
        Returns:
        mne.io.Raw: The reconstructed EEG data after repairing artifacts.
        
        """
        
        events_ica = mne.make_fixed_length_events(self.raw, duration=tstep)
        epochs_ica = mne.Epochs(self.raw, events_ica, tmin=0.0, tmax=tstep, baseline=None, preload=True)
        
        ica = ICA(n_components=n_components, random_state=random_state)
        ica.fit(epochs_ica)
        
        raw_cleaned = self.raw.copy()
        
        # Identify and repair ECG artifacts
        if ecg_ch is not None:
            ecg_epochs = mne.preprocessing.create_ecg_epochs(self.raw, ch_name=ecg_ch)
            ecg_inds, _ = ica.find_bads_ecg(ecg_epochs, ch_name=ecg_ch, method='ctps')
            raw_cleaned = ica.apply(raw_cleaned, exclude=ecg_inds)
            
        # Identify and repair EOG artifacts
        if eog_ch is not None:
            eog_epochs = mne.preprocessing.create_eog_epochs(self.raw, ch_name=eog_ch)
            eog_inds, _ = ica.find_bads_eog(eog_epochs, ch_name=eog_ch, measure='correlation')
            raw_cleaned = ica.apply(raw_cleaned, exclude=eog_inds)
            
            
        # If no ECG or EOG channels were provided, apply ICA without excluding components
        if ecg_ch is None and eog_ch is None:
            raw_cleaned = ica.apply(epochs_ica)
            
        self.raw = raw_cleaned
        
        return raw

 A few things to expand the Preprocessing pipeline further more:
 - The coordinates of the electrodes may vary from each individual, it is important that a normalised space is established for processing.
 - A few more artifacts detection techniques like flat signal, sudden spikes etc. 
 - Including standard artifact detection algorithms.

### Wavelet Transformations

In [4]:
def wavelet_transform(raw, scales, wavelet='morl', plot_scalograms=False):
    
    """
    Apply Continuous Wavelet Transform (CWT) to the EEG data and optionally plot the scalograms.

    Args:
        raw (mne.io.Raw): The raw EEG data.
        scales (list or np.ndarray): The scales range over which to compute the CWT.
        wavelet (str): The wavelet function to use for the CWT. Default is 'morl' (Morlet wavelet).
        plot_scalograms (bool): Whether to plot the scalograms. Default is False.

    Returns:
        np.ndarray: A 2D array containing the scalograms for all EEG channels stacked vertically.
    """
    
    data = raw.get_data()
    ch_names = raw.ch_names
    n_channels = len(ch_names)
    sfreq = raw.info['sfreq']
    
    scalograms = np.empty((np.sum(scales.shape), data.shape[1], n_channels))

    
    for idx, ch_name in enumerate(ch_names):
        
        # Compute the CWT for the current channel
        coef, freqs = pywt.cwt(data[idx], scales, wavelet, sampling_period=1/sfreq)
        scalograms[:, :, idx] = np.abs(coef)

    if plot_scalograms:
        
        fig, axes = plt.subplots(nrows=n_channels, ncols=1, figsize=(15, 3 * n_channels))

        for idx, ch_name in enumerate(ch_names):
            if n_channels == 1:
                ax = axes
            else:
                ax = axes[idx]

            ax.imshow(scalograms[:, :, idx], origin='lower', aspect='auto', extent=[0, data.shape[1] / sfreq, freqs[0], freqs[-1]], cmap='jet')
            ax.set_title(f'Channel: {ch_name}')
            ax.set_xlabel('Time (s)')
            ax.set_ylabel('Frequency (Hz)')

        plt.subplots_adjust(hspace=0.5)
        plt.show()

    return scalograms

### Convolutions with Spatial Pyramid Pooling 

In [5]:
class SpatialPyramidPooling(nn.Module):
    def __init__(self, pool_list):
        super(SpatialPyramidPooling, self).__init__()
        self.pool_list = pool_list

    def forward(self, x):
        num_channels = x.size(1)
        outputs = []

        for num_pool_regions in self.pool_list:
            for jy in range(num_pool_regions):
                
                for ix in range(num_pool_regions):
                    
                    pool_height = max(x.size(2) // num_pool_regions, 1)
                    pool_width = max(x.size(3) // num_pool_regions, 1)
                    
                    
                    x1 = ix * pool_width
                    x2 = ix * pool_width + pool_width
                    y1 = jy * pool_height
                    y2 = jy * pool_height + pool_height
                    

                    x_crop = x[:, :, y1:y2, x1:x2]
                    pooled_val = F.max_pool2d(x_crop, kernel_size=(pool_height, pool_width))
                    outputs.append(pooled_val)

        outputs = torch.cat(outputs, 1)
        return outputs
    


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        
        self.conv1 = nn.Conv2d(in_channels=6, out_channels=36, kernel_size=(3, 3))        
        self.spp = SpatialPyramidPooling(pool_list=[1, 2]) # The data will be split into 1x1 and 2x2 to compute pooling. 
        
    # x represen
    def forward(self, x):
        
        x = x.float()
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        
        x = self.spp(x)
        x = x.view(x.size(0), -1)
        
        return x

### Dataset description: 
This dataset contains behavioral events and intracranial electrophysiological recordings from a categorized free recall task. The experiment consists of participants studying a list of words, presented visually one at a time, completing simple arithmetic problems that function as a distractor, and then freely recalling the words from the just-presented list in any order. (https://openneuro.org/datasets/ds004809/versions/1.0.0)

In [6]:
import os
import pandas as pd
from collections import defaultdict

def load_bipolar_channels(patient_dir, _dir):
    channels_file = os.path.join(patient_dir, f'ses-0/ieeg/{_dir}_ses-0_task-catFR1_acq-bipolar_channels.tsv')
#     print(_dir, channels_file)
    channels = pd.read_csv(channels_file, delimiter='\t')
    return channels['name'].tolist()

def find_common_channels(data_dir):
    _dirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    
    all_channels = defaultdict(int)
    for _dir in _dirs:
        patient_dir = os.path.join(data_dir, _dir)
        channels = load_bipolar_channels(patient_dir, _dir)
        for channel in channels:
            all_channels[channel] += 1
    
    common_channels = [channel for channel, count in all_channels.items() if count == len(_dirs)]
    return common_channels

# Usage
data_dir = '/home/chaithanya/eeg-analysis/ieeg'
common_channels = find_common_channels(data_dir)
print(f"Common channels across patients: {common_channels}")

Common channels across patients: []


- As there are no common channels across patients, First 6 channels from each patient are taken for processing. NOTE: Only a small subset of the dataset is considered for this notebook.

In [7]:
def ieeg_processing(data_dir):
    
    _dirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    
    final_data = []
    
    for _dir in _dirs:
        print("###########")
        print(_dir)
        print("###########")
        
        patient_dir = os.path.join(data_dir, _dir)
        data_path = os.path.join(patient_dir, f'ses-0/ieeg/{_dir}_ses-0_task-catFR1_acq-bipolar_ieeg.edf')
        events_path = os.path.join(patient_dir, f'ses-0/ieeg/{_dir}_ses-0_task-catFR1_events.tsv')
        
        # Load the ieeg data
        ieeg_data = mne.io.read_raw_edf(data_path, preload = True)
        # Load the events.tsv file
        events_df = pd.read_csv(events_path, delimiter='\t')
        
        # Filter the DataFrame to include only rows where trial_type is 'TRAIL'. TRAIL corresponds to a session.
        trials_df = events_df[events_df['trial_type'] == 'TRIAL']
        # Extract the sample numbers from the 'sample' column.
        trial_samples = trials_df['sample'].tolist()
        
        # Initialise the Preprocessing Pipeline and pass the first 2 trails data.
        pipeline = PreprocessingPipeline(ieeg_data.crop(tmin = (trial_samples[0])/(ieeg_data.info['sfreq']), tmax = (trial_samples[2])/(ieeg_data.info['sfreq'])), channels = ieeg_data.ch_names[:6]) # pass the first 6 channels.
        
        # Apply basic filtering techniques
        data = pipeline.filtering(pipeline.raw,l_freq=1, h_freq=30, notch_freq=60)
        # resampling the data
        resampled_data = pipeline.resample(data, 250)
        # repair the artifacts
        clean_data = pipeline.artifact_removal(resampled_data, .99, 42, tstep=1)
        
        
        #################################################################
        # Wavelet Transformations on the artifact free data (clean_data)#
        #################################################################
        
        scales = np.arange(6, 12)
        
        # Continuos wavelet transformation performed on the clean_data. set plot_scalograms = True to plot the scalograms.
        scalograms = wavelet_transform(clean_data, scales, wavelet='morl', plot_scalograms=False) # Output: (height, width, n_channels)
        
        # pytorch accepts (n_channels, height, width)
        scalogram = np.transpose(scalograms, (2, 0, 1))
        
        ##########################################################################
        # Passing the scalograms into the Convolution Layers with Pyramid Pooling#
        ##########################################################################
        
        tensor = torch.tensor(scalogram).unsqueeze(0)
        print(f"Going into the Pyramid pooling with {tensor.shape} shape.")
        myNet = ConvNet()
        features = myNet(tensor)
        eeg_features = features.detach().numpy()
        
        print(f"Coming out of the Pyramid pooling with {eeg_features.shape} shape.")
        
        final_data.append(eeg_features)
        
    return final_data

In [8]:
ieeg_final_data = ieeg_processing('/home/chaithanya/eeg-analysis/ieeg')

###########
sub-R1215M
###########
Extracting EDF parameters from /home/chaithanya/eeg-analysis/ieeg/sub-R1215M/ses-0/ieeg/sub-R1215M_ses-0_task-catFR1_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2904999  =      0.000 ...  2904.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 3301 samples (3.301 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


Not setting metadata
386 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 386 events and 251 original time points ...
0 bad epochs dropped
Fitting ICA to data using 6 channels (please be patient, this may take a while)
Selecting by explained variance: 6 components
Fitting ICA took 2.6s.
Applying ICA to Epochs instance
    Transforming to ICA space (6 components)
    Zeroing out 0 ICA components
    Projecting back using 6 PCA components
Going into the Pyramid pooling with torch.Size([1, 6, 6, 96684]) shape.
Coming out of the Pyramid pooling with (1, 180) shape.
###########
sub-R1169P
###########
Extracting EDF parameters from /home/chaithanya/eeg-analysis/ieeg/sub-R1169P/ses-0/ieeg/sub-R1169P_ses-0_task-catFR1_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3135999  =      0.000 ...  3135.999 secs...
Filtering raw data in 1 contiguous segment
Setti

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s finished


Fitting ICA took 0.4s.
Applying ICA to Epochs instance
    Transforming to ICA space (6 components)
    Zeroing out 0 ICA components
    Projecting back using 6 PCA components
Going into the Pyramid pooling with torch.Size([1, 6, 6, 52466]) shape.
Coming out of the Pyramid pooling with (1, 180) shape.
###########
sub-R1171M
###########
Extracting EDF parameters from /home/chaithanya/eeg-analysis/ieeg/sub-R1171M/ses-0/ieeg/sub-R1171M_ses-0_task-catFR1_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2648999  =      0.000 ...  2648.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz

...

Selecting by explained variance: 5 components
Fitting ICA took 0.2s.
Applying ICA to Epochs instance
    Transforming to ICA space (5 components)
    Zeroing out 0 ICA components
    Projecting back using 6 PCA components
Going into the Pyramid pooling with torch.Size([1, 6, 6, 53920]) shape.
Coming out of the Pyramid pooling with (1, 180) shape.
###########
sub-R1186P
###########
Extracting EDF parameters from /home/chaithanya/eeg-analysis/ieeg/sub-R1186P/ses-0/ieeg/sub-R1186P_ses-0_task-catFR1_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2124999  =      0.000 ...  2124.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband ed

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s finished


Selecting by explained variance: 6 components
Fitting ICA took 1.0s.
Applying ICA to Epochs instance
    Transforming to ICA space (6 components)
    Zeroing out 0 ICA components
    Projecting back using 6 PCA components
Going into the Pyramid pooling with torch.Size([1, 6, 6, 60088]) shape.
Coming out of the Pyramid pooling with (1, 180) shape.
###########
sub-R1093J
###########
Extracting EDF parameters from /home/chaithanya/eeg-analysis/ieeg/sub-R1093J/ses-0/ieeg/sub-R1093J_ses-0_task-catFR1_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2852999  =      0.000 ...  2852.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband ed

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s finished


0 bad epochs dropped
Fitting ICA to data using 6 channels (please be patient, this may take a while)
Selecting by explained variance: 6 components
Fitting ICA took 0.7s.
Applying ICA to Epochs instance
    Transforming to ICA space (6 components)
    Zeroing out 0 ICA components
    Projecting back using 6 PCA components
Going into the Pyramid pooling with torch.Size([1, 6, 6, 51820]) shape.
Coming out of the Pyramid pooling with (1, 180) shape.


In [9]:
final_df = np.vstack((ieeg_final_data))

In [10]:
pd.DataFrame(final_df) # 6 subjects, each subject having 180 colummns/features

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,0.029919,0.067660,0.034211,0.022510,0.000000,0.022176,0.000000,0.000000,0.000000,0.000000,...,0.045003,0.000000,0.015647,0.040237,0.000000,0.000000,0.082744,0.000000,0.000000,0.0
1,0.049005,0.104180,0.084370,0.000000,0.000000,0.000000,0.000000,0.053069,0.123265,0.035456,...,0.000000,0.078702,0.054944,0.108682,0.107503,0.042135,0.027998,0.076549,0.000000,0.0
2,0.095824,0.054917,0.000000,0.020567,0.000000,0.076369,0.000000,0.076956,0.000000,0.093955,...,0.000000,0.000000,0.000000,0.019620,0.000000,0.008420,0.000000,0.000000,0.000000,0.0
3,0.082826,0.000000,0.056198,0.000000,0.000000,0.000000,0.127423,0.048174,0.000000,0.000000,...,0.000000,0.020061,0.130240,0.052694,0.122259,0.000000,0.010513,0.000000,0.000000,0.0
4,0.083243,0.000000,0.000000,0.051758,0.000000,0.000000,0.115557,0.000000,0.000000,0.000000,...,0.065923,0.131522,0.000000,0.010422,0.108738,0.115935,0.000000,0.007636,0.105825,0.0
5,0.000000,0.041942,0.000000,0.000000,0.010177,0.068585,0.045267,0.084446,0.112092,0.000000,...,0.047478,0.000000,0.044606,0.000000,0.115584,0.050228,0.000000,0.050926,0.052760,0.0


### Extending Preprocess pipeline to another dataset

### Dataset description: 
The database contains EEG recordings of subjects before and during the performance of mental arithmetic tasks. In this experiment all subjects are divided into two groups:

Group "G" (24 subjects) performing good quality count (Mean number of operations per 4 minutes = 21, SD = 7.4).

Group "B" (12 subjects) performing bad quality count (Mean number of operations per 4 minutes = 7, SD = 3.6).
(https://www.physionet.org/content/eegmat/1.0.0/)

In [11]:
def eeg_processing(data_path):
    
    total_data = []
    
    # Define the pattern to filter filenames
    pattern = 'Subject*_2.edf'
    # Read the lines from the file
    subject_files = []
    with open(data_path+'RECORDS', 'r') as file:
        for line in file:
            # Strip newline character and check if the line matches the pattern
            if line.strip().endswith(pattern[-5:]):
                subject_files.append(line.strip())
                
    for i in subject_files:
        
        print("################################")
        print(f"processing {i} data")
        print("################################")
        
        data = mne.io.read_raw_edf(data_path+i, preload=True)
        
        # Initialising the Preprocessing pipeline.
        preprocess_pipeline = PreprocessingPipeline(data)
        
        # Apply basic filtering techniques
        filtered_data = preprocess_pipeline.filtering(preprocess_pipeline.raw, l_freq=1, h_freq=30, notch_freq=60)
        # Resampling of the data
        resampled_data = preprocess_pipeline.resample(filtered_data, sfreq=250)
        # Repairing artifacts
        artifact_free_data = preprocess_pipeline.artifact_removal(resampled_data, .99, 42, tstep=1, ecg_ch='ECG ECG', eog_ch=['EEG Fp1', 'EEG Fp2'])
        
        # converting monopolar montage to bi-polar montage.
        re_referenced_data = mne.set_bipolar_reference(artifact_free_data, anode=["EEG Fp1","EEG F7", "EEG T3", "EEG T5", "EEG Fp2", "EEG F8", "EEG T4", "EEG T6", "EEG Fp1", "EEG F3", "EEG C3", "EEG P3", "EEG Fp2", "EEG F4", "EEG C4", "EEG P4", "EEG Fz", "EEG Cz"], cathode=["EEG F7", "EEG T3", "EEG T5", "EEG O1", "EEG F8", "EEG T4", "EEG T6", "EEG O2", "EEG F3", "EEG C3", "EEG P3", "EEG O1", "EEG F4", "EEG C4", "EEG P4", "EEG O2", "EEG Cz", "EEG Pz"], ch_name=['Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1', 'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O2', 'Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'Fz-Cz', 'Cz-Pz'])
        
        re_referenced_data.drop_channels(["EEG A2-A1", "ECG ECG"])
        
        final_data = re_referenced_data.get_data()
        total_data.append(final_data)
        
    return total_data

In [12]:
total_data = eeg_processing("/home/chaithanya/eeg-analysis/data/eeg/")

################################
processing Subject00_2.edf data
################################
Extracting EDF parameters from /home/chaithanya/eeg-analysis/data/eeg/Subject00_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 sec)

Not setting metadata
62 matching events found
No baseline correction applied
0 projection items activated
Using

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.1s finished


Selecting by explained variance: 12 components
Fitting ICA took 0.7s.
Using channel ECG ECG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 2500 samples (10.000 sec)

Number of ECG events detected : 100 (average pulse 96 / min.)
Not setting metadata
100 matching events found
No baseline correction applied
Using data from preloaded Raw for 100 events and 251 original time points ...
1 bad epochs dropped
Using threshold: 0.32 for CTPS ECG detection
Applying ICA to Raw instance
    Transforming to ICA space (12 components)
    Zeroing out 2 ICA components
    

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Applying ICA to Raw instance
    Transforming to ICA space (12 components)
    Zeroing out 0 ICA components
    Projecting back using 21 PCA components
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=18, n_times=15500
    Range : 0 ... 15499 =      0.000 ...    61.996 secs
Ready.
Added the following bipolar channels:
Fp1-F7, F7-T3, T3-T5, T5-O1, Fp2-F8, F8-T4, T4-T6, T6-O2, Fp1-F3, F3-C3, C3-P3, P3-O1, Fp2-F4, F4-C4, C4-P4, P4-O2, Fz-Cz, Cz-Pz
################################
processing Subject01_2.edf data
################################
Extracting EDF parameters from /home/chaithanya/eeg-analysis/data/eeg/Subject01_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandp

...

################################
processing Subject35_2.edf data
################################
Extracting EDF parameters from /home/chaithanya/eeg-analysis/data/eeg/Subject35_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 sec)

Not setting metadata
62 matching events found
No baseline correction applied
0 projection items activated
Using

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.1s finished


Fitting ICA took 1.0s.
Using channel ECG ECG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 2500 samples (10.000 sec)

Number of ECG events detected : 87 (average pulse 84 / min.)
Not setting metadata
87 matching events found
No baseline correction applied
Using data from preloaded Raw for 87 events and 251 original time points ...
1 bad epochs dropped
Using threshold: 0.32 for CTPS ECG detection
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 21 PCA components
Using EOG 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 21 PCA components
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=18, n_times=15500
    Range : 0 ... 15499 =      0.000 ...    61.996 secs
Ready.
Added the following bipolar channels:
Fp1-F7, F7-T3, T3-T5, T5-O1, Fp2-F8, F8-T4, T4-T6, T6-O2, Fp1-F3, F3-C3, C3-P3, P3-O1, Fp2-F4, F4-C4, C4-P4, P4-O2, Fz-Cz, Cz-Pz


In [13]:
total_data = np.asarray(total_data, dtype=np.float32)

In [14]:
total_data.shape

(36, 18, 15500)

The Preprocessed data is fed to the autoencoders to learn the low dimensional representation of the EEG signals. To represent each encoded data as each subject, mean was computed across all channels from each subject.

In [15]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, encoding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_size)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Define the training function
def train_autoencoder(model, data, epochs=10, batch_size=32, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size]
            batch = torch.from_numpy(batch).float()

            optimizer.zero_grad()
            _, decoded = model(batch)
            loss = criterion(decoded, batch)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Perform K-means clustering
def cluster_subjects(encoded_data, n_clusters=2):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(encoded_data)
    return labels

# Example usage
if __name__ == '__main__':
    # Load your EEG data (shape: (n_subjects, n_channels, n_timepoints))
    eeg_data = total_data  # Replace with your actual data
    eeg_data_reshaped = eeg_data.reshape(-1, eeg_data.shape[2])
    scaler = StandardScaler()
    eeg_data_scaled = scaler.fit_transform(eeg_data_reshaped)

    # Define the autoencoder model
    encoding_dim = 32  # Choose the desired encoding dimension
    model = Autoencoder(eeg_data_scaled.shape[1], encoding_dim)

    # Train the autoencoder
    train_autoencoder(model, eeg_data_scaled, epochs=20)

    # Encode each subject's EEG data
    encoded_subjects = []
    for subject in eeg_data:
        encoded_data, _ = model(torch.from_numpy(subject).float())
        encoded_subjects.append(encoded_data.detach().numpy().mean(axis=0))

    # Cluster the subjects based on their encoded EEG data
    encoded_subjects = np.array(encoded_subjects)
    labels = cluster_subjects(encoded_subjects, n_clusters=2)

    # Print the cluster labels for each subject
    for i, label in enumerate(labels):
        print(f'Subject {i+1} cluster label: {label}')

Epoch 1/20, Loss: 0.4134
Epoch 2/20, Loss: 0.4129
Epoch 3/20, Loss: 0.4118
Epoch 4/20, Loss: 0.4105
Epoch 5/20, Loss: 0.4102
Epoch 6/20, Loss: 0.4091
Epoch 7/20, Loss: 0.4071
Epoch 8/20, Loss: 0.4072
Epoch 9/20, Loss: 0.4060
Epoch 10/20, Loss: 0.4055
Epoch 11/20, Loss: 0.4045
Epoch 12/20, Loss: 0.4036
Epoch 13/20, Loss: 0.4042
Epoch 14/20, Loss: 0.4020
Epoch 15/20, Loss: 0.4014
Epoch 16/20, Loss: 0.3998
Epoch 17/20, Loss: 0.3994
Epoch 18/20, Loss: 0.3977
Epoch 19/20, Loss: 0.3996
Epoch 20/20, Loss: 0.3976
Subject 1 cluster label: 0
Subject 2 cluster label: 1
Subject 3 cluster label: 0
Subject 4 cluster label: 0
Subject 5 cluster label: 0
Subject 6 cluster label: 1
Subject 7 cluster label: 0
Subject 8 cluster label: 0
Subject 9 cluster label: 0
Subject 10 cluster label: 0
Subject 11 cluster label: 0
Subject 12 cluster label: 0
Subject 13 cluster label: 1
Subject 14 cluster label: 0
Subject 15 cluster label: 0
Subject 16 cluster label: 0
Subject 17 cluster label: 0
Subject 18 cluster lab